## POS tagging using modified Viterbi

### Data Preparation

In [1]:
#Importing libraries
import nltk
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from collections import Counter
import time
from nltk.tokenize import word_tokenize
from functools import reduce
import random
import pdb
import re

In [2]:
# Reading Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
# splitting Treebank dataset into train & validation set in ration of 95:5 %
train_set, validation_set = train_test_split(nltk_data,train_size=.95,test_size=0.05,random_state=70)

In [4]:
print("Dataset size-> nltk: {}, Training:{}, Validation: {}".format(len(nltk_data),len(train_set),len(validation_set)))

Dataset size-> nltk: 3914, Training:3718, Validation: 196


In [5]:
tagged_train_set=[tup for set in train_set for tup in set]
# Extracging vocabulary and respective Tags sperately
vocabulary=(set([v[0] for v in tagged_train_set]))
tags=set(v[1] for v in tagged_train_set)
print("Length-> Vocabulary: {}, Tags: {}".format(len(vocabulary),len(tags)))
print("Available Tags-> ",tags)

Length-> Vocabulary: 12088, Tags: 12
Available Tags->  {'VERB', 'NUM', 'CONJ', 'DET', 'ADV', 'NOUN', 'PRT', 'ADP', 'X', 'ADJ', '.', 'PRON'}


### Build the vanilla Viterbi based POS tagger

In [6]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = tagged_train_set):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [7]:
def t2_given_t1(t2, t1, train_bag = tagged_train_set):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [8]:
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [9]:
# tags_matrix data Frame
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))

In [10]:
tags_df.loc['.', :]

VERB    0.089807
NUM     0.081275
CONJ    0.058015
DET     0.173507
ADV     0.053345
NOUN    0.222452
PRT     0.002515
ADP     0.091334
X       0.027211
ADJ     0.043197
.       0.091783
PRON    0.065469
Name: ., dtype: float32

In [11]:
def Viterbi(words, train_bag = tagged_train_set,backoff=[]):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        e=[]
        t=[]
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]

            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            t.append(transition_p)
            e.append(emission_p) #Adding Emission prbability for all tag
            state_probability = emission_p * transition_p    
            p.append(state_probability)
        pmax = max(p)
        state_max = None
        if pmax==0.0:
            linker=backoff.copy()
            while linker!=[]:
                if state_max==None:
                    state_max=linker.pop()([word])[0][1]
                else:
                    linker.clear()
            if state_max==None and backoff!=[]:
                if sum(e)==0.0:
                    state_max = T[t.index(max(t))]
        # getting state for which probability is maximum
        if state_max==None:
            state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [12]:
validation_run_base=[tup for sent in validation_set for tup in sent] # it's a list of tuple with WORD and TAG

validation_tagged_words = [tup[0] for sent in validation_set for tup in sent] # list of words only

In [13]:
#Validation set (word,tag)
tagged_words_validation=[tup for ls in validation_set for tup in ls]

#Validation set words
vocabulary_validation=set([t[0] for t in tagged_words_validation])
print("-"*100)
print("Total unique vocab")
print("Training set->{} Validation set->{}\n".format(str(len(vocabulary)),str(len(vocabulary_validation))))

#Unknown words
print("-"*100)
print("Vocabulary only in Validation Set (not in Training set)")
UnknownWords=list(vocabulary_validation-vocabulary)
print("Total Unknown words: {}, sample->{}\n".format(len(UnknownWords),UnknownWords[0:3]))

#Common Vocabulary
print("-"*100)
CommonWords=list(vocabulary_validation.intersection(vocabulary))
print("Vocabulary common in Training & Validation Set")
print("Total common words: {}, sample->{}".format(len(CommonWords),CommonWords[0:3]))

----------------------------------------------------------------------------------------------------
Total unique vocab
Training set->12088 Validation set->1820

----------------------------------------------------------------------------------------------------
Vocabulary only in Validation Set (not in Training set)
Total Unknown words: 320, sample->['Mexican', '7.8', 'Miguel']

----------------------------------------------------------------------------------------------------
Vocabulary common in Training & Validation Set
Total common words: 1500, sample->['terms', 'program', 'authority']


### Solve the problem of unknown words

In [14]:
UnknownWords[0:10]

['Mexican',
 '7.8',
 'Miguel',
 'diagnosed',
 '12.09',
 'weight',
 'Hayes',
 'deliberating',
 'livelihood',
 'sue']

In [15]:
# This is a rule based tagger that will apply the most common cardinal rule to identify Number and X (unknown) formed due
# to numbers or special characters like $,*.
# Examples: Street-50, **T**, 237, 236,000
def cardinal_X_tagger(word):
    patterns = [
    (r'^[aA-zZ].*[0-9]+','NOUN'),  #Flat/Door Number, Street Number
    (r'^(0|([*|-|$].*))','X'), #Any special form of number like *T* *a-767, 0
    (r'[0-9].?[,\/]?[0-9]*','NUM'),# Any Form of number    
    ]
    regextagger=nltk.RegexpTagger(patterns)
    return regextagger.tag(word)

In [16]:
# This is a rule based tagger that will apply the most common morpholigical rules of English
# Rule 1: Anything formed with a pattern num-alphabets will be Adjective example 230-seats
# Rule 2: Anything with all capital letters and important Salutations will be treated as Noun ex: Mr. Mrs. Md, CIA
# Rule 3: Anything ending with 'ed' or 'ing' or 'es' will be considered as verb example running, walked
# Rule 4: Anything ending with s or 's will be considered as Noun example 
# Rule 5: Anything with starting letter as capital followed by all small letters will be treated as Noun ex: Testimony, Sir, Mrs
def english_morphological_tagger(word):
    patterns = [
    (r'^([0-9]|[aA-zZ])+\-[aA-zZ]*$','ADJ'), #adjective like 100-megabytes 237-Seats
    (r'^[A-Z]+([a-z]{1,2})?\.?$','NOUN'),# Capitalization rule of English and Salutation
    (r'[aA-zZ]+(\'s|s)$', 'NOUN'),             # possessive nouns & plural nouns
    (r'^[A-Z]{1}[a-z]*$','NOUN'),
    (r'[aA-zZ]+(ed|ing|es)$', 'VERB')
    ]
    regextagger=nltk.RegexpTagger(patterns)
    status=regextagger.tag(word)
    return status

In [17]:
# cardinal_X_tagger(['0.28'])
# english_morphological_tagger(['Soviet'])
tagged_seq = Viterbi(['Soviet'],backoff=[cardinal_X_tagger,english_morphological_tagger])
tagged_seq

[('Soviet', 'ADJ')]

#### Evaluating tagging accuracy

> Vanila Viterbi Algorithm

In [18]:
# tagging the test sentences
tagged_seq = Viterbi(validation_tagged_words)

In [19]:
check = [i for i, j in zip(tagged_seq, validation_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print(accuracy)
incorrect_tagged_cases = [j for i, j in enumerate(zip(tagged_seq, validation_run_base)) if j[0]!=j[1]]

0.9171868497711194


In [20]:
incorrect_tagged_cases

[(('resumption', 'VERB'), ('resumption', 'NOUN')),
 (('shuttle', 'VERB'), ('shuttle', 'NOUN')),
 (('expendable', 'VERB'), ('expendable', 'ADJ')),
 (('launch-vehicle', 'VERB'), ('launch-vehicle', 'NOUN')),
 (('engines', 'VERB'), ('engines', 'NOUN')),
 (('two-letter', 'VERB'), ('two-letter', 'ADJ')),
 (('consonant', 'VERB'), ('consonant', 'ADJ')),
 (('sounds', 'VERB'), ('sounds', 'NOUN')),
 (('sounds', 'VERB'), ('sounds', 'NOUN')),
 (('to', 'PRT'), ('to', 'ADP')),
 (('exclusion', 'VERB'), ('exclusion', 'NOUN')),
 (('sounds', 'VERB'), ('sounds', 'NOUN')),
 (('Heiwado', 'VERB'), ('Heiwado', 'NOUN')),
 (('18.3', 'VERB'), ('18.3', 'NUM')),
 (('Continental', 'VERB'), ('Continental', 'NOUN')),
 (('Baking', 'VERB'), ('Baking', 'NOUN')),
 (('bread', 'VERB'), ('bread', 'NOUN')),
 (('excess', 'ADJ'), ('excess', 'NOUN')),
 (('percent', 'VERB'), ('percent', 'NOUN')),
 (('newsstand', 'VERB'), ('newsstand', 'NOUN')),
 (('freeway', 'VERB'), ('freeway', 'NOUN')),
 (('inevitable', 'VERB'), ('inevitable',

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

> Modified Vanila Viterbi Algorithm
> > The two modification taggers cardinal_X_tagger,english_morphological_tagger passed and applied in chaining

  - Note: A modification in Verbitri algorithm, if Emission probability of all Tags is 0 than we will consider only Transition probability

In [21]:
tagged_seq = Viterbi(validation_tagged_words,backoff=[cardinal_X_tagger,english_morphological_tagger])

In [22]:
check = [i for i, j in zip(tagged_seq, validation_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print(accuracy)
incorrect_tagged_cases = [j for i, j in enumerate(zip(tagged_seq, validation_run_base)) if j[0]!=j[1]]

0.9506866416978776


In [23]:
incorrect_tagged_cases

[(('expendable', 'DET'), ('expendable', 'ADJ')),
 (('launch-vehicle', 'ADJ'), ('launch-vehicle', 'NOUN')),
 (('consonant', 'NOUN'), ('consonant', 'ADJ')),
 (('sounds', 'VERB'), ('sounds', 'NOUN')),
 (('sounds', 'VERB'), ('sounds', 'NOUN')),
 (('to', 'PRT'), ('to', 'ADP')),
 (('sounds', 'VERB'), ('sounds', 'NOUN')),
 (('bread', 'DET'), ('bread', 'NOUN')),
 (('excess', 'ADJ'), ('excess', 'NOUN')),
 (('inevitable', 'VERB'), ('inevitable', 'ADJ')),
 (('places', 'NOUN'), ('places', 'VERB')),
 (('appoint', 'NOUN'), ('appoint', 'VERB')),
 (('passers-by', 'ADJ'), ('passers-by', 'NOUN')),
 (('down', 'ADV'), ('down', 'ADP')),
 (('including', 'VERB'), ('including', 'ADP')),
 (('restricts', 'NOUN'), ('restricts', 'VERB')),
 (('inauspicious', 'NOUN'), ('inauspicious', 'ADJ')),
 (('beginning', 'VERB'), ('beginning', 'NOUN')),
 (('over', 'ADP'), ('over', 'PRT')),
 (('Christian', 'ADJ'), ('Christian', 'NOUN')),
 (('executive', 'NOUN'), ('executive', 'ADJ')),
 (('off', 'PRT'), ('off', 'ADP')),
 (('fina

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [27]:
# CASE 1 : Number is tagged by Vanilla Viterbi is Adverb, with modified it is correctly as Num
print("Before {}, After{}".format(Viterbi(['8.06']),Viterbi(['8.06'],backoff=[cardinal_X_tagger,english_morphological_tagger])))
print("Before {}, After{}".format(Viterbi(['CERTIFICATES']),Viterbi(['CERTIFICATES'],backoff=[cardinal_X_tagger,english_morphological_tagger])))
print("Before {}, After{}".format(Viterbi(['parts-engineering']),Viterbi(['parts-engineering'],backoff=[cardinal_X_tagger,english_morphological_tagger])))
print("Before {}, After{}".format(Viterbi(['Consent']),Viterbi(['Consent'],backoff=[cardinal_X_tagger,english_morphological_tagger])))
print("Before {}, After{}".format(Viterbi(['100-megabytes']),Viterbi(['100-megabytes'],backoff=[cardinal_X_tagger,english_morphological_tagger])))

Before [('8.06', 'VERB')], After[('8.06', 'NUM')]
Before [('CERTIFICATES', 'VERB')], After[('CERTIFICATES', 'NOUN')]
Before [('parts-engineering', 'VERB')], After[('parts-engineering', 'ADJ')]
Before [('Consent', 'VERB')], After[('Consent', 'NOUN')]
Before [('100-megabytes', 'VERB')], After[('100-megabytes', 'ADJ')]
